## ANN
- keras를 이용한 ANN 모델 구축

### 데이터 준비

In [1]:
import glob
base_path = './xray_classification_pneumonia/Dataset_compressed/train'
file_path = list(glob.glob(base_path + "/*/*.*"))

pneumonia = list(glob.glob(base_path+"/PNEUMONIA/*.*"))
normal = list(glob.glob(base_path+"/NORMAL/*.*"))

In [2]:
len(pneumonia), len(normal)

(3875, 1341)

In [3]:
import matplotlib.pyplot as plt
import os

test = os.path.dirname(file_path[0])
print(test)
class_name = os.path.split(os.path.dirname(test))
print(class_name)

./xray_classification_pneumonia/Dataset_compressed/train\NORMAL
('./xray_classification_pneumonia/Dataset_compressed', 'train')


In [4]:
# folder name to label
labels = []

for fp in file_path:
    tmp = os.path.dirname(fp)
    class_name = os.path.split(tmp)
    if class_name[1] == "PNEUMONIA":
        labels.append(1)
    else:
        labels.append(0)

In [5]:
import numpy as np
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2


img_width = 60
img_height = 60

dataset = np.ndarray(shape=(len(file_path), img_height*img_width), dtype=np.float32)

i=0
for _file in file_path:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (img_width, img_height))
    
    dataset[i] = img_resized.flatten()
    i += 1
    if i % 250 == 0:
        print(f"{i} images to array")
print("Done!")

250 images to array
500 images to array
750 images to array
1000 images to array
1250 images to array
1500 images to array
1750 images to array
2000 images to array
2250 images to array
2500 images to array
2750 images to array
3000 images to array
3250 images to array
3500 images to array
3750 images to array
4000 images to array
4250 images to array
4500 images to array
4750 images to array
5000 images to array
Done!


In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

In [9]:
len(X_train), len(X_test), len(y_train), len(y_test)

(4172, 1044, 4172, 1044)

In [10]:
y_train, y_test = map(lambda x: to_categorical(x), [y_train, y_test])

### 모델 생성

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

In [14]:
def build():
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(3600,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    return model

model = build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               360100    
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 2)                 102       
                                                                 
Total params: 365,252
Trainable params: 365,252
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.predict(dataset)

163/163 [==============================] - 1s 4ms/step


array([[2.0962368e-28, 9.9999994e-01],
       [1.0479997e-21, 9.9999994e-01],
       [0.0000000e+00, 9.9999994e-01],
       ...,
       [4.6451209e-20, 9.9999994e-01],
       [0.0000000e+00, 9.9999994e-01],
       [7.6988302e-16, 9.9999994e-01]], dtype=float32)

### Adam 학습

In [17]:
model_adam = build()

model_adam.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model_adam.fit(
    X_train, y_train, validation_data=(X_test, y_test), batch_size=64, epochs=20
)

Epoch 1/20
66/66 [==============================] - 1s 9ms/step - loss: 31.7285 - accuracy: 0.7951 - val_loss: 1.9673 - val_accuracy: 0.9320
Epoch 2/20
66/66 [==============================] - 0s 4ms/step - loss: 5.1577 - accuracy: 0.8615 - val_loss: 2.9879 - val_accuracy: 0.8918
Epoch 3/20
66/66 [==============================] - 0s 3ms/step - loss: 2.8799 - accuracy: 0.9017 - val_loss: 3.9332 - val_accuracy: 0.8784
Epoch 4/20
66/66 [==============================] - 0s 3ms/step - loss: 1.7164 - accuracy: 0.9358 - val_loss: 1.3544 - val_accuracy: 0.9464
Epoch 5/20
66/66 [==============================] - 0s 3ms/step - loss: 2.0907 - accuracy: 0.9221 - val_loss: 1.3967 - val_accuracy: 0.9511
Epoch 6/20
66/66 [==============================] - 0s 5ms/step - loss: 1.1916 - accuracy: 0.9453 - val_loss: 1.4169 - val_accuracy: 0.9387
Epoch 7/20
66/66 [==============================] - 0s 4ms/step - loss: 0.8503 - accuracy: 0.9506 - val_loss: 1.4485 - val_accuracy: 0.9339
Epoch 8/20
66/66 [=

### 예측

In [18]:
import matplotlib.pyplot as plt
import numpy as np

predictions = model_adam.predict(X_test)
predicted_class = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

33/33 [==============================] - 0s 3ms/step
